# Exploratory Data Analysis

Importação, unificação e limpeza das Bases de Dados

In [20]:
# Carregar dados localmente (adaptado do Google Colab)
import os
print("Carregando dados do Atlas Brasil...")


Carregando dados do Atlas Brasil...


In [22]:
# Configuração do PySpark
import findspark
findspark.init()

from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Inicializar Spark Session com configurações otimizadas
spark = SparkSession.builder \
    .appName("PrevisaoDemandaSaude") \
    .config("spark.sql.adaptive.enabled", "true") \
    .config("spark.sql.adaptive.coalescePartitions.enabled", "true") \
    .config("spark.sql.adaptive.skewJoin.enabled", "true") \
    .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
    .config("spark.sql.execution.arrow.pyspark.enabled", "true") \
    .config("spark.driver.memory", "2g") \
    .config("spark.driver.maxResultSize", "1g") \
    .getOrCreate()

print("Spark Session iniciada com sucesso!")
print(f"Versão do Spark: {spark.version}")

# Carregar dados com pandas (mais estável para Excel)
print("Carregando dados do Atlas Brasil...")
ivs_pandas = pd.read_excel("../data/raw/Atlasvis_DadosBrutos.xlsx")
print(f"Dados carregados com pandas: {ivs_pandas.shape[0]} registros, {ivs_pandas.shape[1]} variáveis")

# Usar pandas para processamento inicial (mais estável)
print("Usando pandas para processamento inicial...")
ivs = ivs_pandas
print(f"DataFrame pandas: {ivs.shape[0]} registros, {ivs.shape[1]} variáveis")

Spark Session iniciada com sucesso!
Versão do Spark: 3.5.1
Carregando dados do Atlas Brasil...
Dados carregados com pandas: 340786 registros, 103 variáveis
Usando pandas para processamento inicial...
DataFrame pandas: 340786 registros, 103 variáveis


## 📊 Análise Exploratória de Dados (EDA) Robusta

### Objetivos da EDA:
1. **Compreender a distribuição** dos dados
2. **Identificar padrões** e tendências temporais
3. **Detectar outliers** e valores atípicos
4. **Analisar relacionamentos** entre variáveis
5. **Validar qualidade** dos dados para análises futuras


In [ ]:
# Configuração para visualizações
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 10

# Configuração para exibir todas as colunas
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

print("✅ Configurações de visualização aplicadas!")


### 1. Análise Temporal dos Dados

Vamos começar analisando como os dados estão distribuídos ao longo do tempo e identificar padrões temporais.


In [ ]:
# Análise temporal dos dados
print("=== ANÁLISE TEMPORAL ===")
print(f"Período dos dados: {ivs['ano'].min()} - {ivs['ano'].max()}")
print(f"Anos únicos: {sorted(ivs['ano'].unique())}")

# Contagem de registros por ano
contagem_por_ano = ivs['ano'].value_counts().sort_index()
print(f"\nRegistros por ano:")
print(contagem_por_ano)

# Visualização da distribuição temporal
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Gráfico 1: Contagem de registros por ano
axes[0,0].bar(contagem_por_ano.index, contagem_por_ano.values, color='skyblue', alpha=0.7)
axes[0,0].set_title('Distribuição de Registros por Ano', fontsize=14, fontweight='bold')
axes[0,0].set_xlabel('Ano')
axes[0,0].set_ylabel('Número de Registros')
axes[0,0].grid(True, alpha=0.3)

# Gráfico 2: Evolução temporal (linha)
axes[0,1].plot(contagem_por_ano.index, contagem_por_ano.values, marker='o', linewidth=2, markersize=8)
axes[0,1].set_title('Evolução Temporal dos Registros', fontsize=14, fontweight='bold')
axes[0,1].set_xlabel('Ano')
axes[0,1].set_ylabel('Número de Registros')
axes[0,1].grid(True, alpha=0.3)

# Gráfico 3: Distribuição percentual
percentual = (contagem_por_ano / contagem_por_ano.sum() * 100).round(2)
axes[1,0].pie(percentual.values, labels=percentual.index, autopct='%1.1f%%', startangle=90)
axes[1,0].set_title('Distribuição Percentual por Ano', fontsize=14, fontweight='bold')

# Gráfico 4: Boxplot da distribuição temporal
axes[1,1].boxplot([ivs[ivs['ano'] == ano]['ano'].values for ano in contagem_por_ano.index], 
                  labels=contagem_por_ano.index)
axes[1,1].set_title('Distribuição dos Anos (Boxplot)', fontsize=14, fontweight='bold')
axes[1,1].set_xlabel('Ano')
axes[1,1].set_ylabel('Valores')

plt.tight_layout()
plt.show()

# Estatísticas temporais
print(f"\n=== ESTATÍSTICAS TEMPORAIS ===")
print(f"Média de registros por ano: {contagem_por_ano.mean():.0f}")
print(f"Mediana de registros por ano: {contagem_por_ano.median():.0f}")
print(f"Desvio padrão: {contagem_por_ano.std():.0f}")
print(f"Ano com mais registros: {contagem_por_ano.idxmax()} ({contagem_por_ano.max():,} registros)")
print(f"Ano com menos registros: {contagem_por_ano.idxmin()} ({contagem_por_ano.min():,} registros)")


### 2. Análise de Valores Ausentes (Missing Values)

Vamos analisar de forma detalhada onde estão os valores ausentes e como isso impacta nossa análise.


In [ ]:
# Análise detalhada de valores ausentes
print("=== ANÁLISE DE VALORES AUSENTES ===")

# Calcular valores ausentes por coluna
missing_data = ivs.isnull().sum()
missing_percent = (missing_data / len(ivs)) * 100

# Criar DataFrame com informações de missing
missing_df = pd.DataFrame({
    'Coluna': missing_data.index,
    'Valores_Ausentes': missing_data.values,
    'Percentual_Ausente': missing_percent.values
}).sort_values('Valores_Ausentes', ascending=False)

# Filtrar apenas colunas com valores ausentes
missing_df = missing_df[missing_df['Valores_Ausentes'] > 0]

print(f"Total de colunas com valores ausentes: {len(missing_df)}")
print(f"Total de colunas no dataset: {len(ivs.columns)}")
print(f"Percentual de colunas com missing: {(len(missing_df)/len(ivs.columns)*100):.1f}%")

# Visualização dos valores ausentes
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Gráfico 1: Top 15 colunas com mais valores ausentes
top_missing = missing_df.head(15)
axes[0,0].barh(range(len(top_missing)), top_missing['Valores_Ausentes'], color='coral', alpha=0.7)
axes[0,0].set_yticks(range(len(top_missing)))
axes[0,0].set_yticklabels(top_missing['Coluna'], fontsize=8)
axes[0,0].set_title('Top 15 Colunas com Mais Valores Ausentes', fontsize=12, fontweight='bold')
axes[0,0].set_xlabel('Número de Valores Ausentes')
axes[0,0].grid(True, alpha=0.3)

# Gráfico 2: Percentual de valores ausentes
axes[0,1].barh(range(len(top_missing)), top_missing['Percentual_Ausente'], color='lightblue', alpha=0.7)
axes[0,1].set_yticks(range(len(top_missing)))
axes[0,1].set_yticklabels(top_missing['Coluna'], fontsize=8)
axes[0,1].set_title('Percentual de Valores Ausentes (Top 15)', fontsize=12, fontweight='bold')
axes[0,1].set_xlabel('Percentual (%)')
axes[0,1].grid(True, alpha=0.3)

# Gráfico 3: Heatmap de valores ausentes (amostra)
# Selecionar apenas algumas colunas para o heatmap (muitas colunas tornam o gráfico ilegível)
colunas_importantes = ['ivs', 'idhm', 'ivs_infraestrutura_urbana', 'ivs_capital_humano', 
                      'ivs_renda_e_trabalho', 'idhm_educ', 'idhm_renda', 'idhm_long']
colunas_existentes = [col for col in colunas_importantes if col in ivs.columns]

if colunas_existentes:
    # Amostra de 1000 registros para o heatmap
    sample_data = ivs[colunas_existentes].sample(n=min(1000, len(ivs)), random_state=42)
    sns.heatmap(sample_data.isnull(), cbar=True, yticklabels=False, ax=axes[1,0], 
                cmap='viridis', alpha=0.8)
    axes[1,0].set_title('Heatmap de Valores Ausentes (Amostra)', fontsize=12, fontweight='bold')
    axes[1,0].set_xlabel('Variáveis')

# Gráfico 4: Distribuição de valores ausentes por ano
missing_por_ano = ivs.groupby('ano').apply(lambda x: x.isnull().sum().sum())
axes[1,1].bar(missing_por_ano.index, missing_por_ano.values, color='lightgreen', alpha=0.7)
axes[1,1].set_title('Total de Valores Ausentes por Ano', fontsize=12, fontweight='bold')
axes[1,1].set_xlabel('Ano')
axes[1,1].set_ylabel('Total de Valores Ausentes')
axes[1,1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Estatísticas de missing
print(f"\n=== ESTATÍSTICAS DE VALORES AUSENTES ===")
print(f"Total de valores ausentes no dataset: {missing_data.sum():,}")
print(f"Percentual total de valores ausentes: {(missing_data.sum() / (len(ivs) * len(ivs.columns)) * 100):.2f}%")
print(f"Coluna com mais valores ausentes: {missing_df.iloc[0]['Coluna']} ({missing_df.iloc[0]['Valores_Ausentes']:,} valores)")
print(f"Coluna com menos valores ausentes: {missing_df.iloc[-1]['Coluna']} ({missing_df.iloc[-1]['Valores_Ausentes']:,} valores)")

# Análise por tipo de variável
print(f"\n=== ANÁLISE POR TIPO DE VARIÁVEL ===")
for dtype in ivs.dtypes.unique():
    colunas_tipo = ivs.select_dtypes(include=[dtype]).columns
    missing_tipo = ivs[colunas_tipo].isnull().sum().sum()
    total_tipo = len(colunas_tipo) * len(ivs)
    print(f"{dtype}: {missing_tipo:,} valores ausentes ({missing_tipo/total_tipo*100:.1f}%)")


### 3. Análise das Variáveis Numéricas Principais

Agora vamos focar nas variáveis mais importantes para o TCC e analisar suas distribuições.


In [ ]:
# Selecionar variáveis numéricas principais para análise
variaveis_principais = ['ivs', 'idhm', 'ivs_infraestrutura_urbana', 'ivs_capital_humano', 
                       'ivs_renda_e_trabalho', 'idhm_educ', 'idhm_renda', 'idhm_long']

# Verificar quais variáveis existem no dataset
variaveis_existentes = [var for var in variaveis_principais if var in ivs.columns]
print(f"Variáveis principais encontradas: {variaveis_existentes}")

# Filtrar apenas registros com dados válidos para essas variáveis
dados_validos = ivs[variaveis_existentes].dropna()
print(f"\nRegistros com dados válidos para todas as variáveis principais: {len(dados_validos):,}")
print(f"Percentual de registros válidos: {len(dados_validos)/len(ivs)*100:.1f}%")

# Análise descritiva detalhada
print(f"\n=== ESTATÍSTICAS DESCRITIVAS DETALHADAS ===")
desc_stats = dados_validos.describe()
print(desc_stats.round(4))

# Análise de distribuição
fig, axes = plt.subplots(3, 3, figsize=(18, 15))
axes = axes.ravel()

for i, var in enumerate(variaveis_existentes):
    if i < len(axes):
        # Histograma
        axes[i].hist(dados_validos[var], bins=50, alpha=0.7, color='skyblue', edgecolor='black')
        axes[i].set_title(f'Distribuição de {var}', fontweight='bold')
        axes[i].set_xlabel(var)
        axes[i].set_ylabel('Frequência')
        axes[i].grid(True, alpha=0.3)
        
        # Adicionar estatísticas no gráfico
        mean_val = dados_validos[var].mean()
        median_val = dados_validos[var].median()
        axes[i].axvline(mean_val, color='red', linestyle='--', alpha=0.8, label=f'Média: {mean_val:.3f}')
        axes[i].axvline(median_val, color='green', linestyle='--', alpha=0.8, label=f'Mediana: {median_val:.3f}')
        axes[i].legend(fontsize=8)

# Remover subplots vazios
for i in range(len(variaveis_existentes), len(axes)):
    fig.delaxes(axes[i])

plt.tight_layout()
plt.show()

# Análise de outliers usando IQR
print(f"\n=== ANÁLISE DE OUTLIERS (Método IQR) ===")
for var in variaveis_existentes:
    Q1 = dados_validos[var].quantile(0.25)
    Q3 = dados_validos[var].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    outliers = dados_validos[(dados_validos[var] < lower_bound) | (dados_validos[var] > upper_bound)]
    outlier_percent = len(outliers) / len(dados_validos) * 100
    
    print(f"{var}:")
    print(f"  - Limites: [{lower_bound:.3f}, {upper_bound:.3f}]")
    print(f"  - Outliers: {len(outliers):,} ({outlier_percent:.1f}%)")
    print(f"  - Valores extremos: min={dados_validos[var].min():.3f}, max={dados_validos[var].max():.3f}")
    print()


### 4. Análise de Correlação entre Variáveis

Vamos analisar como as variáveis se relacionam entre si através de correlações.


In [ ]:
# Análise de correlação
print("=== ANÁLISE DE CORRELAÇÃO ===")

# Calcular matriz de correlação
correlation_matrix = dados_validos.corr()
print(f"Matriz de correlação calculada para {len(variaveis_existentes)} variáveis")

# Visualização da matriz de correlação
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Gráfico 1: Heatmap de correlação
mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))
sns.heatmap(correlation_matrix, mask=mask, annot=True, cmap='coolwarm', center=0,
            square=True, linewidths=0.5, cbar_kws={"shrink": 0.8}, ax=axes[0,0])
axes[0,0].set_title('Matriz de Correlação (Heatmap)', fontsize=14, fontweight='bold')

# Gráfico 2: Correlação com IVS
if 'ivs' in variaveis_existentes:
    ivs_corr = correlation_matrix['ivs'].drop('ivs').sort_values(key=abs, ascending=False)
    colors = ['red' if x < 0 else 'blue' for x in ivs_corr.values]
    axes[0,1].barh(range(len(ivs_corr)), ivs_corr.values, color=colors, alpha=0.7)
    axes[0,1].set_yticks(range(len(ivs_corr)))
    axes[0,1].set_yticklabels(ivs_corr.index)
    axes[0,1].set_title('Correlação com IVS', fontsize=14, fontweight='bold')
    axes[0,1].set_xlabel('Coeficiente de Correlação')
    axes[0,1].axvline(0, color='black', linestyle='-', alpha=0.3)
    axes[0,1].grid(True, alpha=0.3)

# Gráfico 3: Correlação com IDHM
if 'idhm' in variaveis_existentes:
    idhm_corr = correlation_matrix['idhm'].drop('idhm').sort_values(key=abs, ascending=False)
    colors = ['red' if x < 0 else 'blue' for x in idhm_corr.values]
    axes[1,0].barh(range(len(idhm_corr)), idhm_corr.values, color=colors, alpha=0.7)
    axes[1,0].set_yticks(range(len(idhm_corr)))
    axes[1,0].set_yticklabels(idhm_corr.index)
    axes[1,0].set_title('Correlação com IDHM', fontsize=14, fontweight='bold')
    axes[1,0].set_xlabel('Coeficiente de Correlação')
    axes[1,0].axvline(0, color='black', linestyle='-', alpha=0.3)
    axes[1,0].grid(True, alpha=0.3)

# Gráfico 4: Scatter plot IVS vs IDHM
if 'ivs' in variaveis_existentes and 'idhm' in variaveis_existentes:
    # Amostra para melhor visualização
    sample_size = min(5000, len(dados_validos))
    sample_data = dados_validos.sample(n=sample_size, random_state=42)
    
    scatter = axes[1,1].scatter(sample_data['ivs'], sample_data['idhm'], 
                               alpha=0.6, s=20, c=sample_data['idhm'], cmap='viridis')
    axes[1,1].set_xlabel('IVS (Índice de Vulnerabilidade Social)')
    axes[1,1].set_ylabel('IDHM (Índice de Desenvolvimento Humano Municipal)')
    axes[1,1].set_title('IVS vs IDHM (Scatter Plot)', fontsize=14, fontweight='bold')
    axes[1,1].grid(True, alpha=0.3)
    
    # Adicionar linha de tendência
    z = np.polyfit(sample_data['ivs'], sample_data['idhm'], 1)
    p = np.poly1d(z)
    axes[1,1].plot(sample_data['ivs'], p(sample_data['ivs']), "r--", alpha=0.8, linewidth=2)
    
    # Adicionar colorbar
    plt.colorbar(scatter, ax=axes[1,1], label='IDHM')

plt.tight_layout()
plt.show()

# Análise de correlações mais fortes
print(f"\n=== CORRELAÇÕES MAIS FORTES ===")
# Criar lista de pares de correlação
corr_pairs = []
for i in range(len(correlation_matrix.columns)):
    for j in range(i+1, len(correlation_matrix.columns)):
        var1 = correlation_matrix.columns[i]
        var2 = correlation_matrix.columns[j]
        corr_value = correlation_matrix.iloc[i, j]
        corr_pairs.append((var1, var2, corr_value))

# Ordenar por valor absoluto da correlação
corr_pairs.sort(key=lambda x: abs(x[2]), reverse=True)

print("Top 10 correlações mais fortes:")
for i, (var1, var2, corr) in enumerate(corr_pairs[:10]):
    direction = "positiva" if corr > 0 else "negativa"
    strength = "forte" if abs(corr) > 0.7 else "moderada" if abs(corr) > 0.5 else "fraca"
    print(f"{i+1:2d}. {var1} ↔ {var2}: {corr:.3f} (correlação {strength} {direction})")

# Análise de correlações significativas
print(f"\n=== CORRELAÇÕES SIGNIFICATIVAS (|r| > 0.5) ===")
strong_correlations = [(var1, var2, corr) for var1, var2, corr in corr_pairs if abs(corr) > 0.5]
for var1, var2, corr in strong_correlations:
    direction = "positiva" if corr > 0 else "negativa"
    print(f"• {var1} ↔ {var2}: {corr:.3f} (correlação {direction})")


### 5. Análise Temporal das Variáveis Principais

Vamos analisar como as variáveis evoluem ao longo do tempo (2000, 2010, 2020).


### 6. Resumo e Insights da EDA

Vamos consolidar os principais achados da análise exploratória.


In [ ]:
# Resumo consolidado da EDA
print("="*80)
print("📊 RESUMO CONSOLIDADO DA ANÁLISE EXPLORATÓRIA DE DADOS")
print("="*80)

print(f"\n🔍 INFORMAÇÕES GERAIS DO DATASET:")
print(f"   • Total de registros: {len(ivs):,}")
print(f"   • Total de variáveis: {len(ivs.columns)}")
print(f"   • Período temporal: {ivs['ano'].min()} - {ivs['ano'].max()}")
print(f"   • Anos com dados completos: 2000, 2010, 2020")

print(f"\n📈 COBERTURA DE DADOS:")
print(f"   • Registros com dados válidos para variáveis principais: {len(dados_validos):,}")
print(f"   • Percentual de cobertura: {len(dados_validos)/len(ivs)*100:.1f}%")
print(f"   • Total de valores ausentes: {ivs.isnull().sum().sum():,}")

print(f"\n📊 VARIÁVEIS PRINCIPAIS ANALISADAS:")
for i, var in enumerate(variaveis_existentes, 1):
    if var in dados_validos.columns:
        mean_val = dados_validos[var].mean()
        std_val = dados_validos[var].std()
        min_val = dados_validos[var].min()
        max_val = dados_validos[var].max()
        print(f"   {i}. {var.upper()}:")
        print(f"      Média: {mean_val:.3f} | Desvio: {std_val:.3f} | Range: [{min_val:.3f}, {max_val:.3f}]")

print(f"\n🔗 CORRELAÇÕES MAIS IMPORTANTES:")
if 'corr_pairs' in locals():
    for i, (var1, var2, corr) in enumerate(corr_pairs[:5], 1):
        direction = "positiva" if corr > 0 else "negativa"
        strength = "forte" if abs(corr) > 0.7 else "moderada" if abs(corr) > 0.5 else "fraca"
        print(f"   {i}. {var1} ↔ {var2}: {corr:.3f} (correlação {strength} {direction})")

print(f"\n📅 EVOLUÇÃO TEMPORAL (2000 → 2010 → 2020):")
if 'dados_temporais' in locals():
    for var in ['ivs', 'idhm']:
        if var in dados_temporais.columns:
            medias_por_ano = dados_temporais.groupby('ano')[var].mean()
            print(f"   {var.upper()}:")
            for ano in [2000, 2010, 2020]:
                if ano in medias_por_ano.index:
                    print(f"      {ano}: {medias_por_ano[ano]:.3f}")

print(f"\n⚠️  PRINCIPAIS DESAFIOS IDENTIFICADOS:")
print(f"   • Alta proporção de valores ausentes em várias variáveis")
print(f"   • Distribuição desigual dos dados entre os anos")
print(f"   • Necessidade de tratamento de outliers em algumas variáveis")
print(f"   • Limitação temporal (apenas 3 anos com dados completos)")

print(f"\n✅ PONTOS FORTES DO DATASET:")
print(f"   • Variáveis socioeconômicas relevantes para análise de saúde")
print(f"   • Cobertura nacional (todos os municípios brasileiros)")
print(f"   • Indicadores padronizados (IVS, IDHM)")
print(f"   • Dados de qualidade para os anos selecionados")

print(f"\n🎯 PRÓXIMOS PASSOS RECOMENDADOS:")
print(f"   1. Tratamento de valores ausentes (imputação ou exclusão)")
print(f"   2. Análise geográfica (por região/estado)")
print(f"   3. Criação de variáveis derivadas (índices compostos)")
print(f"   4. Análise de clusters para identificar padrões regionais")
print(f"   5. Preparação dos dados para modelagem preditiva")

print(f"\n📋 QUALIDADE DOS DADOS PARA ANÁLISE:")
print(f"   • Dados válidos suficientes para análises estatísticas: ✅")
print(f"   • Variáveis numéricas adequadamente formatadas: ✅")
print(f"   • Cobertura temporal adequada para análise longitudinal: ✅")
print(f"   • Variáveis relevantes para previsão de demanda em saúde: ✅")

print("\n" + "="*80)
print("🏁 EDA CONCLUÍDA COM SUCESSO!")
print("="*80)


In [23]:
# Informações gerais do DataFrame
print("=== INFORMAÇÕES GERAIS ===")
print(f"Dimensões: {ivs.shape[0]} registros, {ivs.shape[1]} variáveis")
print(f"Tipos de dados:")
print(ivs.dtypes.value_counts())

print("\n=== VALORES AUSENTES ===")
valores_ausentes = ivs.isnull().sum().sort_values(ascending=False)
print(valores_ausentes[valores_ausentes > 0].head(10))

print("\n=== ESTATÍSTICAS DESCRITIVAS ===")
# Apenas variáveis numéricas para describe
variaveis_numericas = ivs.select_dtypes(include=[np.number]).columns
print(ivs[variaveis_numericas].describe())


=== INFORMAÇÕES GERAIS ===
Dimensões: 340786 registros, 103 variáveis
Tipos de dados:
object     99
int64       2
float64     2
Name: count, dtype: int64

=== VALORES AUSENTES ===
prosp_soc    200628
ivs          200474
idhm          63342
dtype: int64

=== ESTATÍSTICAS DESCRITIVAS ===
                  id            ano            ivs           idhm
count  340786.000000  340786.000000  140312.000000  277444.000000
mean   190671.979342    2005.582078       0.370811       0.608250
std    127660.960189       5.531609       0.137952       0.123754
min         1.000000    2000.000000       0.034000       0.000000
25%     95079.250000    2000.000000       0.266000       0.533000
50%    180275.500000    2010.000000       0.357000       0.620000
75%    265471.750000    2010.000000       0.467000       0.697000
max    660006.000000    2022.000000       0.872000       0.965000


Estrutura da base:

A base possui 340.786 observações (linhas) e 103 variáveis (colunas).

Entre essas variáveis, 99 estão no formato object, o que indica que seus valores estão armazenados como texto (muito provavelmente com vírgulas no lugar de pontos decimais), o que impede análises numéricas até que sejam convertidas.

Presença de valores ausentes:

Identificamos que várias variáveis importantes possuem muitos valores nulos (NaN).

Por exemplo:

A variável ivs (Índice de Vulnerabilidade Social) aparece em apenas 140.312 das 340.786 linhas.

A variável idhm (Índice de Desenvolvimento Humano Municipal) está presente em 277.444 linhas, ainda assim com uma quantidade considerável de ausências.

Resumo estatístico preliminar:

A função describe() permitiu observar que os valores de ivs variam entre 0.034 e 0.872, e o idhm varia entre 0.000 e 0.965.

As médias de ivs e idhm foram de aproximadamente 0.37 e 0.61, respectivamente, o que indica padrões médios de vulnerabilidade e desenvolvimento moderados, com uma ampla variação entre os territórios analisados.

Será necessário:

Selecionar apenas as colunas relevantes para o TCC;

Converter os dados de texto para números reais (float);

Tratar os valores ausentes antes de qualquer unificação com outras bases.

In [24]:
# Selecionando variáveis importantes para o TCC
colunas_usar = [
    'ano', 'municipio',
    'ivs', 'ivs_infraestrutura_urbana', 'ivs_capital_humano', 'ivs_renda_e_trabalho',
    'idhm', 'idhm_educ', 'idhm_renda', 'idhm_long',
    't_vulner', 't_desocup18m', 't_formal_18m', 't_analf_15m', 't_renda_trab'
]

# Filtrar apenas colunas que existem no DataFrame
colunas_existentes = [col for col in colunas_usar if col in ivs.columns]
print(f"Colunas selecionadas: {colunas_existentes}")

ivs_filt = ivs[colunas_existentes].copy()
print(f"DataFrame filtrado: {ivs_filt.shape[0]} registros, {ivs_filt.shape[1]} variáveis")
ivs_filt.head()



Colunas selecionadas: ['ano', 'municipio', 'ivs', 'ivs_infraestrutura_urbana', 'ivs_capital_humano', 'ivs_renda_e_trabalho', 'idhm', 'idhm_educ', 'idhm_renda', 'idhm_long', 't_vulner', 't_desocup18m', 't_formal_18m', 't_analf_15m', 't_renda_trab']
DataFrame filtrado: 340786 registros, 15 variáveis


,ano,municipio,ivs,ivs_infraestrutura_urbana,ivs_capital_humano,ivs_renda_e_trabalho,idhm,idhm_educ,idhm_renda,idhm_long,t_vulner,t_desocup18m,t_formal_18m,t_analf_15m,t_renda_trab
0,2000,geral,0.335,"0,285","0,400","0,321",0.703,"0,569","0,740","0,827","37,11","12,16","57,36","8,32","77,51"
1,2010,geral,0.255,"0,251","0,267","0,248",0.784,"0,708","0,791","0,860","21,40","5,89","65,13","5,76","73,56"
2,2011,geral,0.214,"0,197","0,228","0,215",0.790,"0,746","0,762","0,867","19,50","5,26","63,50","5,26","78,16"
3,2012,geral,0.200,"0,199","0,216","0,185",0.800,"0,754","0,777","0,875","15,61","4,75","64,65","5,27","77,34"
4,2013,geral,0.197,"0,200","0,210","0,182",0.806,"0,764","0,782","0,878","15,34","4,91","65,85","5,18","77,68"


Após a análise exploratória inicial, o próximo passo foi filtrar apenas as colunas mais importantes para os objetivos do TCC. Essa etapa tem como objetivo:

1- Reduzir o tamanho da base,

2- Focar nas variáveis que ajudam a explicar a demanda por serviços de atenção primária,

3- E facilitar os próximos passos de limpeza, unificação e análise.

As variáveis selecionadas foram:

| Variável                                       | Significado básico                                                                  |
| ---------------------------------------------- | ----------------------------------------------------------------------------------- |
| `ano`                                          | Ano de referência dos dados                                                         |
| `municipio`                                    | Código do município (ou geral, no caso de agregações)                               |
| `ivs`                                          | Índice de Vulnerabilidade Social geral                                              |
| `ivs_infraestrutura_urbana`                    | Subíndice de infraestrutura (água, esgoto, coleta de lixo etc.)                     |
| `ivs_capital_humano`                           | Subíndice ligado a educação e saúde                                                 |
| `ivs_renda_e_trabalho`                         | Subíndice de renda e inserção no mercado de trabalho                                |
| `idhm`, `idhm_educ`, `idhm_renda`, `idhm_long` | Dimensões do Índice de Desenvolvimento Humano (geral, educação, renda, longevidade) |
| `t_vulner`                                     | Proporção da população com renda per capita ≤ ½ salário mínimo                      |
| `t_desocup18m`                                 | Taxa de desocupação entre pessoas com 18 anos ou mais                               |
| `t_formal_18m`                                 | Percentual de trabalhadores formais com 18 anos ou mais                             |
| `t_analf_15m`                                  | Taxa de analfabetismo entre maiores de 15 anos                                      |
| `t_renda_trab`                                 | Percentual da renda domiciliar proveniente do trabalho                              |


Próximo passo: converter todas as colunas para o tipo float

In [25]:
# Converter tipos de dados no pandas
print("Convertendo tipos de dados...")

# Definir colunas numéricas (excluindo ano e municipio)
colunas_numericas = [col for col in ivs_filt.columns if col not in ['ano', 'municipio']]

# Aplicar conversões
for col in colunas_numericas:
    if ivs_filt[col].dtype == 'object':
        # Substituir vírgulas por pontos
        ivs_filt[col] = ivs_filt[col].str.replace(',', '.', regex=False)
        # Converter para numérico
        ivs_filt[col] = pd.to_numeric(ivs_filt[col], errors='coerce')

print("Conversão de tipos concluída!")
print(f"Tipos de dados: {ivs_filt.dtypes.value_counts()}")

Convertendo tipos de dados...
Conversão de tipos concluída!
Tipos de dados: float64    13
int64       1
object      1
Name: count, dtype: int64


In [26]:
# Verificar tipos de dados após conversão
print("Tipos de dados após conversão:")
print(ivs_filt.dtypes)


Tipos de dados após conversão:
ano                            int64
municipio                     object
ivs                          float64
ivs_infraestrutura_urbana    float64
ivs_capital_humano           float64
ivs_renda_e_trabalho         float64
idhm                         float64
idhm_educ                    float64
idhm_renda                   float64
idhm_long                    float64
t_vulner                     float64
t_desocup18m                 float64
t_formal_18m                 float64
t_analf_15m                  float64
t_renda_trab                 float64
dtype: object


In [27]:
# Ver quantos valores faltantes ainda existem por coluna
print("Valores ausentes por coluna:")
valores_ausentes = ivs_filt.isnull().sum().sort_values(ascending=False)
print(valores_ausentes[valores_ausentes > 0])


Valores ausentes por coluna:
ivs                          200474
ivs_capital_humano           162454
ivs_infraestrutura_urbana    130687
idhm                          63342
idhm_long                     63342
idhm_educ                     63339
ivs_renda_e_trabalho          58653
t_formal_18m                  55410
t_renda_trab                  55299
idhm_renda                    55126
t_desocup18m                  54979
t_vulner                      54758
t_analf_15m                   54331
dtype: int64


Após a seleção das variáveis relevantes, realizamos a conversão dos dados do formato textual (com vírgulas) para valores numéricos (float), utilizando o método str.replace(',', '.') seguido de pd.to_numeric().

Essa conversão foi necessária porque as colunas estavam originalmente no tipo object, o que inviabilizava qualquer tipo de análise estatística ou modelagem. Agora, todas as variáveis quantitativas estão corretamente no formato float64.

Com os dados convertidos, rodamos um diagnóstico com isnull().sum() para verificar a presença de valores ausentes (NaN).

Os resultados mostraram que:

Muitas colunas possuem dezenas de milhares de valores ausentes.

Por exemplo:

ivs: ~200.000 ausentes (de 340.786 linhas)

ivs_capital_humano: ~162.000 ausentes

idhm: ~63.000 ausentes

Apenas ano e municipio não têm valores nulos.

Proximos passos:

Filtrar para anos em que há muitos dados válidos, manter os anos com maior cobertura e qualidade de dados, podemos verificar quais anos possuem menos nulos e usar apenas esses

In [28]:
# Verificar quantos valores válidos (não nulos) existem por variável em cada ano
print("Contagem de registros por ano:")
print(ivs_filt.groupby("ano").count().sort_index(ascending=True))


Contagem de registros por ano:
      municipio    ivs  ivs_infraestrutura_urbana  ivs_capital_humano  \
ano                                                                     
2000     161480  63172                      96302               80934   
2010     163299  71045                     104652               89364   
2011       1161    514                        772                 668   
2012       1161    516                        774                 666   
2013       1161    516                        774                 674   
2014       1161    516                        774                 681   
2015       1161    516                        774                 682   
2016       1458    502                        754                 665   
2017       1458    502                        752                 663   
2018       1455    504                        756                 663   
2019       1458    504                        755                 670   
2020       1458    5

Análise dos resultados
🟢 2000 e 2010

São os únicos anos com cobertura ampla para quase todas as variáveis:

ivs: ~63k (2000) / ~71k (2010)

idhm: ~130k (2000) / ~139k (2010)

t_vulner: ~131k (2000) / ~140k (2010)


🔴 2011–2022

Só ~1000 registros por ano.

Ruins para análise estatística confiável.


Com base na análise da cobertura dos dados ano a ano, optamos por manter apenas os anos 2000, 2010 e 2020. Essa decisão foi tomada por três razões principais:

1-Cobertura completa das principais variáveis (como IVS, IDHM e indicadores de vulnerabilidade e trabalho) nesses anos.

2-Possibilidade de observar a evolução dos determinantes sociais da saúde em um intervalo de 20 anos, com saltos decenais.

3-Alinhamento com ciclos censitários e de políticas públicas, facilitando análises longitudinais e comparativas.

In [29]:
# Filtrar os anos desejados
anos_para_manter = [2000, 2010, 2020]
ivs_filtrado = ivs_filt[ivs_filt['ano'].isin(anos_para_manter)].copy()

# Verificar tamanho da nova base
print(f"Base filtrada por anos: {ivs_filtrado.shape[0]} registros, {ivs_filtrado.shape[1]} variáveis")


Base filtrada por anos: 326237 registros, 15 variáveis


In [30]:
# Remover linhas com valores ausentes restantes
ivs_filtrado = ivs_filtrado.dropna()

# Confirmar o novo tamanho da base
print(f"Base após remoção de valores ausentes: {ivs_filtrado.shape[0]} registros, {ivs_filtrado.shape[1]} variáveis")


Base após remoção de valores ausentes: 134556 registros, 15 variáveis


Com isso, temos agora uma base filtrada, limpa e pronta para análise, contendo:

134.556 linhas

15 variáveis relevantes

Somente dos anos 2000, 2010 e 2020

E sem valores nulos (NaN)

Com isso, o DataFrame final passou a conter 134.556 registros completos, distribuídos entre os 3 anos selecionados, com 15 variáveis socioeconômicas e de desenvolvimento humano.


Proximo passo: Unificar com outras bases

Integrar com dados demográficos por bairro, distrito ou setor

Explorar como variáveis locais afetam os indicadores

In [31]:
# Carregar dados de bairros
print("Carregando dados de bairros...")


Carregando dados de bairros...


In [32]:
# Carregar a base de bairros
bairros = pd.read_excel("../data/raw/Agregados_por_bairros_basico_BR.xlsx")

print(f"Dados de bairros carregados: {bairros.shape[0]} registros, {bairros.shape[1]} variáveis")
print("\nPrimeiras linhas:")
bairros.head()


Dados de bairros carregados: 17576 registros, 30 variáveis

Primeiras linhas:


,CD_BAIRRO,NM_BAIRRO,CD_REGIAO,NM_REGIAO,CD_UF,NM_UF,CD_MUN,NM_MUN,CD_DIST,NM_DIST,...,CD_CONCURB,NM_CONCURB,AREA_KM2,v0001,v0002,v0003,v0004,v0005,v0006,v0007
0,1100015001,Centro,1,Norte,11,Rondônia,1100015,Alta Floresta D'Oeste,110001505,Alta Floresta D'Oeste,...,.,NaN,0.628243,1213,637,633,4,2.5,0.0146,480
1,1100015002,Liberdade,1,Norte,11,Rondônia,1100015,Alta Floresta D'Oeste,110001505,Alta Floresta D'Oeste,...,.,NaN,0.951916,1029,488,488,0,2.6,0.0281,391
2,1100015003,Cidade Alta,1,Norte,11,Rondônia,1100015,Alta Floresta D'Oeste,110001505,Alta Floresta D'Oeste,...,.,NaN,0.596322,1288,602,600,2,2.6,0.004,499
3,1100015005,Princesa Isabel,1,Norte,11,Rondônia,1100015,Alta Floresta D'Oeste,110001505,Alta Floresta D'Oeste,...,.,NaN,1.951955,2846,1242,1242,0,2.7,0.0179,1060
4,1100015006,Redondo,1,Norte,11,Rondônia,1100015,Alta Floresta D'Oeste,110001505,Alta Floresta D'Oeste,...,.,NaN,1.453434,2365,990,990,0,2.7,0.0619,873


Colunas relevantes para o merge:

| Coluna do DataFrame de bairros | Significado                                          |
| ------------------------------ | ---------------------------------------------------- |
| `CD_MUN`                       | Código do município (ex: 1100015) – **fundamental!** |
| `NM_MUN`                       | Nome do município (ex: Floresta D’Oeste)             |
| `NM_BAIRRO`                    | Nome do bairro – útil para análise local             |
| `v0001`, `v0002`, ...          | Variáveis demográficas agregadas por bairro          |


a base ivs_filtrado, a coluna municipio é equivalente ao CD_MUN da base de bairros.
Isso significa que podemos fazer o merge pelas colunas:

ivs_filtrado["municipio"]

bairros["CD_MUN"]


ATENÇÃO: Na base ivs_filtrado, a coluna municipio está no formato object e pode conter "geral" ou estar formatada diferente.

Proximo passo: Vamos converter tudo para string e preparar o merge com segurança.

In [33]:
# Converter códigos para string para garantir correspondência
ivs_filtrado["municipio"] = ivs_filtrado["municipio"].astype(str)
bairros["CD_MUN"] = bairros["CD_MUN"].astype(str)

# Fazer o merge (junção) com base no código do município
ivs_com_bairros = pd.merge(ivs_filtrado, bairros, left_on="municipio", right_on="CD_MUN", how="inner")

# Verificar o resultado
ivs_com_bairros.head()


,ano,municipio,ivs,ivs_infraestrutura_urbana,ivs_capital_humano,ivs_renda_e_trabalho,idhm,idhm_educ,idhm_renda,idhm_long,...,CD_CONCURB,NM_CONCURB,AREA_KM2,v0001,v0002,v0003,v0004,v0005,v0006,v0007
0,2000,5217401,0.234,0.104,0.327,0.271,0.686,0.537,0.718,0.836,...,.,NaN,0.504719,1635,812,807,5,2.3,0.0074,680
1,2000,5217401,0.234,0.104,0.327,0.271,0.686,0.537,0.718,0.836,...,.,NaN,0.944571,2645,1175,1175,0,2.5,0.0028,1058
2,2000,5217401,0.234,0.104,0.327,0.271,0.686,0.537,0.718,0.836,...,.,NaN,0.225690,962,398,398,0,2.6,0,369
3,2000,5217401,0.234,0.104,0.327,0.271,0.686,0.537,0.718,0.836,...,.,NaN,0.276035,1198,483,480,3,2.7,0.0091,439
4,2000,5217401,0.234,0.104,0.327,0.271,0.686,0.537,0.718,0.836,...,.,NaN,0.944546,2946,1275,1271,4,2.6,0.0035,1138


Para enriquecer a base principal com informações geodemográficas, realizamos a junção da base ivs_filtrado com a base Agregados_por_bairros_basico_BR.xlsx, utilizando como chave comum o código do município (CD_MUN).

Esse merge permitiu incorporar informações complementares como:

Nome dos bairros e municípios,

Dados de área territorial (AREA_KM2),

Variáveis populacionais codificadas .

Esse cruzamento torna a base muito mais robusta para análises espaciais, demográficas e comparativas, especialmente no contexto da demanda por serviços de atenção primária à saúde.

Proximo passo: Carregar a base de distritos

In [34]:
# Carregar dados de distritos
print("Carregando dados de distritos...")


Carregando dados de distritos...


In [35]:
# Carregar a base de distritos
distritos = pd.read_excel("../data/raw/Agregados_por_distritos_demografia_BR.xlsx")

print(f"Dados de distritos carregados: {distritos.shape[0]} registros, {distritos.shape[1]} variáveis")
print("\nPrimeiras linhas:")
distritos.head()


Dados de distritos carregados: 10698 registros, 38 variáveis

Primeiras linhas:


,CD_DIST,NM_DIST,V01006,V01007,V01008,V01009,V01010,V01011,V01012,V01013,...,V01032,V01033,V01034,V01035,V01036,V01037,V01038,V01039,V01040,V01041
0,110001505,Alta Floresta D'Oeste,16699,8267,8432,592,598,589,645,612,...,1166,1144,1250,1191,1211,2485,2487,2028,1479,1053
1,110001515,Filadélfia d'Oeste,551,299,252,23,27,25,19,19,...,39,46,44,44,33,79,84,65,48,19
2,110001520,Izidolândia,532,287,245,25,18,17,25,12,...,44,42,42,34,44,85,78,62,37,22
3,110001525,Nova Gease d'Oeste,1071,562,509,35,32,43,57,27,...,90,79,104,73,92,126,120,130,82,54
4,110001530,Rolim de Moura do Guaporé,801,442,359,42,31,45,45,31,...,55,86,81,51,58,129,116,85,43,23


Estrutura da base de distritos

| Coluna            | Significado presumido                                                                   |
| ----------------- | --------------------------------------------------------------------------------------- |
| `CD_DIST`         | Código do distrito                                                                      |
| `NM_DIST`         | Nome do distrito                                                                        |
| `V01006`–`V01041` | Variáveis demográficas (como população, idade, etc.) – usar dicionário para interpretar |


Essa base não possui explicitamente o código do município (CD_MUN), mas o código do distrito (CD_DIST) começa com o código do município.

Exemplo:

CD_DIST = 110001505 → o código do município é 1100015



Estratégia para unir com a base ivs_filtrado

1-Vamos extrair os 7 primeiros dígitos de CD_DIST como CD_MUN

2-Converter ambos os códigos para string

3-Fazer o merge com ivs_filtrado usando municipio == CD_MUN

In [36]:
# Extrair o código do município dos 7 primeiros dígitos de CD_DIST
distritos["CD_MUN"] = distritos["CD_DIST"].astype(str).str[:7]

# Converter tipo para garantir merge
ivs_filtrado["municipio"] = ivs_filtrado["municipio"].astype(str)
distritos["CD_MUN"] = distritos["CD_MUN"].astype(str)

# Fazer o merge
ivs_com_distritos = pd.merge(ivs_filtrado, distritos, left_on="municipio", right_on="CD_MUN", how="inner")

# Verificar o resultado
ivs_com_distritos.head()

,ano,municipio,ivs,ivs_infraestrutura_urbana,ivs_capital_humano,ivs_renda_e_trabalho,idhm,idhm_educ,idhm_renda,idhm_long,...,V01033,V01034,V01035,V01036,V01037,V01038,V01039,V01040,V01041,CD_MUN
0,2000,5300108,0.314,0.422,0.291,0.229,0.805,0.687,0.857,0.886,...,182575,207834,225156,220704,457174,460525,330213,205840,158123,5300108
1,2010,5300108,0.254,0.408,0.188,0.167,0.885,0.806,0.920,0.935,...,182575,207834,225156,220704,457174,460525,330213,205840,158123,5300108
2,2000,5200134,0.340,0.236,0.490,0.293,0.654,0.493,0.699,0.812,...,1448,1627,1562,1539,3195,3267,2758,1821,1331,5200134
3,2010,5200134,0.262,0.243,0.293,0.251,0.729,0.629,0.737,0.835,...,1448,1627,1562,1539,3195,3267,2758,1821,1331,5200134
4,2010,5200159,0.280,0.255,0.204,0.382,0.734,0.706,0.684,0.821,...,129,129,119,119,304,321,359,310,249,5200159


Para enriquecer a análise dos indicadores sociais com dados em nível de distrito, realizamos a união entre a base principal (ivs_filtrado) e a base Agregados_por_distritos_demografia_BR.xlsx, com base no código do município (CD_MUN), extraído dos sete primeiros dígitos do campo CD_DIST.

A união permitiu incorporar variáveis agregadas por distrito (ex: população por faixa etária, escolaridade, etc.), que serão úteis para analisar padrões de vulnerabilidade, desigualdade e demandas sociais na atenção primária à saúde.

Essa base unificada será uma das principais ferramentas do projeto para identificar correlações entre condições demográficas e indicadores de desenvolvimento, além de suportar análises de previsão de demanda por serviços públicos.


Proximo passo: Adicionar a ultima base de dados Agregados_por_setores_demografia_BR.xlsx

In [37]:
# Carregar dados de setores censitários
print("Carregando dados de setores censitários...")


Carregando dados de setores censitários...


In [38]:
# Carregar a base de setores censitários
setores = pd.read_excel("../data/raw/Agregados_por_setores_demografia_BR.xlsx")

print(f"Dados de setores carregados: {setores.shape[0]} registros, {setores.shape[1]} variáveis")
print("\nPrimeiras linhas:")
setores.head()


Dados de setores carregados: 458772 registros, 37 variáveis

Primeiras linhas:


,CD_setor,V01006,V01007,V01008,V01009,V01010,V01011,V01012,V01013,V01014,...,V01032,V01033,V01034,V01035,V01036,V01037,V01038,V01039,V01040,V01041
0,110001505000002,928,428,500,30,39,24,44,36,25,...,68,62,88,68,58,144,129,124,66,53
1,110001505000003,556,270,286,15,20,20,19,30,20,...,36,47,38,47,44,83,95,48,47,34
2,110001505000004,222,108,114,5,4,9,14,7,5,...,11,15,26,11,11,33,37,25,23,17
3,110001505000006,785,408,377,36,33,34,41,42,27,...,63,61,75,71,53,124,92,88,57,40
4,110001505000007,748,373,375,28,25,33,27,24,31,...,52,54,52,51,61,110,116,93,63,46


O que observamos da base setores:

| Coluna              | Interpretação                                                      |
| ------------------- | ------------------------------------------------------------------ |
| `CD_setor`          | Código único do setor censitário (15 dígitos)                      |
| `V01006` a `V01041` | Variáveis demográficas (idade, renda, etc. – checar no dicionário) |


Sabemos que:

Os 7 primeiros dígitos do CD_setor representam o município → compatível com ivs_filtrado["municipio"]

Os 9 primeiros dígitos representam o distrito → compatível com CD_DIST da base de distritos

Próximo passo: Integrar a base fazendo

1-Extrair os 7 primeiros dígitos de CD_setor para formar o campo CD_MUN

2-Unir com a base principal ivs_filtrado pela coluna municipio

3-Assim, teremos a base IVS/IDHM com dados de setores censitários incluídos

UTILIZEI TODA A RAM DISPONIVEL, ISSO FAZ COM QUE AO TENTAR RODAR O CODIGO PARA UNIR TODAS AS BASES, A SESSÃO FALHE E REINICIE, ASSIM TENDO QUE REINICIAR OS DADOS PARA IMPORTAÇÃO DE VARIAVEL E BASE DE DADOS

In [42]:
# Preparar dados de setores para merge
print("Preparando dados de setores...")
setores["CD_setor"] = setores["CD_setor"].astype(str)
setores["CD_MUN"] = setores["CD_setor"].str[:7]
ivs_filtrado["municipio"] = ivs_filtrado["municipio"].astype(str)

# Filtrar setores apenas para municípios que temos no IVS
print("Filtrando setores para municípios do IVS...")
municipios_ivs = set(ivs_filtrado["municipio"].unique())
setores_filtrados = setores[setores["CD_MUN"].isin(municipios_ivs)].copy()

print(f"Setores filtrados: {setores_filtrados.shape[0]} registros")

# Se ainda for muito grande, usar apenas uma amostra
if len(setores_filtrados) > 50000:
    print("Muitos setores! Usando amostra de 50.000 registros...")
    setores_filtrados = setores_filtrados.sample(n=50000, random_state=42)
    print(f"Setores após amostragem: {setores_filtrados.shape[0]} registros")

# Fazer merge com setores filtrados
print("Fazendo merge com dados de setores...")
try:
    ivs_com_setores = pd.merge(ivs_filtrado, setores_filtrados, left_on="municipio", right_on="CD_MUN", how="inner")
    print(f"Base final com setores: {ivs_com_setores.shape[0]} registros, {ivs_com_setores.shape[1]} variáveis")
except MemoryError:
    print("Erro de memória! Usando apenas dados do IVS sem setores...")
    # Usar apenas os dados do IVS se der erro de memória
    ivs_com_setores = ivs_filtrado.copy()
    print(f"Base final (apenas IVS): {ivs_com_setores.shape[0]} registros, {ivs_com_setores.shape[1]} variáveis")

print("\nPrimeiras linhas da base unificada:")
ivs_com_setores.head()


Preparando dados de setores...
Filtrando setores para municípios do IVS...
Setores filtrados: 458574 registros
Muitos setores! Usando amostra de 50.000 registros...
Setores após amostragem: 50000 registros
Fazendo merge com dados de setores...
Base final com setores: 16709286 registros, 53 variáveis

Primeiras linhas da base unificada:


,ano,municipio,ivs,ivs_infraestrutura_urbana,ivs_capital_humano,ivs_renda_e_trabalho,idhm,idhm_educ,idhm_renda,idhm_long,...,V01033,V01034,V01035,V01036,V01037,V01038,V01039,V01040,V01041,CD_MUN
0,2000,5300108,0.314,0.422,0.291,0.229,0.805,0.687,0.857,0.886,...,80,60,69,92,290,210,81,37,14,5300108
1,2000,5300108,0.314,0.422,0.291,0.229,0.805,0.687,0.857,0.886,...,12,7,16,48,122,49,23,19,10,5300108
2,2000,5300108,0.314,0.422,0.291,0.229,0.805,0.687,0.857,0.886,...,55,63,80,96,263,152,160,69,43,5300108
3,2000,5300108,0.314,0.422,0.291,0.229,0.805,0.687,0.857,0.886,...,26,24,29,28,51,68,50,31,34,5300108
4,2000,5300108,0.314,0.422,0.291,0.229,0.805,0.687,0.857,0.886,...,28,39,44,47,124,97,77,74,25,5300108


In [44]:
# Salvar dados processados
print("Salvando dados processados...")
ivs_com_setores.to_csv("../data/processed/dados_unificados.csv", index=False)
print("Dados salvos em: ../data/processed/dados_unificados.csv")

# Resumo final
print(f"\n=== RESUMO FINAL ===")
print(f"Base unificada: {ivs_com_setores.shape[0]:,} registros e {ivs_com_setores.shape[1]} variáveis")
print(f"Anos incluídos: {sorted(ivs_com_setores['ano'].unique())}")
print(f"Valores ausentes: {ivs_com_setores.isnull().sum().sum()}")

# Fechar Spark Session
spark.stop()
print("\nSpark Session encerrada!")


Salvando dados processados...
Dados salvos em: ../data/processed/dados_unificados.csv

=== RESUMO FINAL ===
Base unificada: 16,709,286 registros e 53 variáveis
Anos incluídos: [np.int64(2000), np.int64(2010)]
Valores ausentes: 0

Spark Session encerrada!
